In [1]:
import discord
import yaml

import os
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

# Obtention du chemin absolu vers le répertoire de travail actuel
current_dir = os.getcwd()

# Chemin relatif vers le fichier YAML
api_keys_path = os.path.join(current_dir, "../private/api-keys.yaml")

async def main(api_keys_path):
    # Charger les clés d'API depuis le fichier YAML
    with open(api_keys_path, "r") as file:
        secrets = yaml.safe_load(file)
        bot_token = secrets["discord_bot_token"]
        text_analytics_key = secrets["text_analytics_key"]
        text_analytics_endpoint = secrets["text_analytics_endpoint"]

    # Définition des intents
    intents = discord.Intents.default()
    intents.guilds = True
    intents.messages = True
    intents.guild_messages = True  # Activation spécifique pour les messages de guildes
    intents.members = True
    intents.message_content = True

    client = discord.Client(intents=intents)

    # Initialisation du client Text Analytics
    credential = AzureKeyCredential(text_analytics_key)
    text_analytics_client = TextAnalyticsClient(endpoint=text_analytics_endpoint, credential=credential)

    @client.event
    async def on_message(message):
        message_content = str(message.content)
        message_author = str(message.author)
        message_name = str(message.author.mention)
        print(f'New message ->{message_content}')

        # Recherche des canaux "test-bot-g1", "test-bot-g2", "test-bot-g3", et "test-bot-g4" dans le serveur
        test_channels = []
        for guild in client.guilds:
            for channel in guild.channels:
                if channel.name in ["test-bot-g1", "test-bot-g2", "test-bot-g3", "test-bot-g4"]:
                    test_channels.append(channel)

        # Si le message est dans l'un des canaux de test
        if message.channel in test_channels:
            texte = message_content

            # Analyse de sentiment
            resultat_sentiment = text_analytics_client.analyze_sentiment(documents=[texte])[0]
            sentiment = resultat_sentiment.sentiment
            print("Sentiment : ", sentiment)

            # Détection de la langue
            resultat_langue = text_analytics_client.detect_language(documents=[texte])[0]
            langue = resultat_langue.primary_language.name
            print("Langue : ", langue)

            # Analyse des entités nommées
            resultat_entites = text_analytics_client.recognize_entities(documents=[texte])[0]
            entites = [entite.text for entite in resultat_entites.entities]
            print("Entités nommées : ", entites)

            # Recherche du canal "report-bot-g2" dans le serveur
            for guild in client.guilds:
                for channel in guild.channels:
                    if channel.name == "report-bot-g2":
                        report_channel = channel
                        break

            # Envoi de l'alerte
            if (message_author != 'Bully Macguire#2062' and channel.name == "report-bot-g2" and message_author != 'Arsene Deghela#3390' and message_author != 'MessageBot#9405'):
                await report_channel.send(f'{message_name} : {message_content}\nLangue : {langue}\nSentiment : {sentiment}\nEntités nommées : {entites}')

    await client.start(bot_token)

# Exécution du bot
await main(api_keys_path)


New message ->Hello
Sentiment :  neutral
Langue :  English
Entités nommées :  []
New message -><@779045486476263445> : Hello
Langue : English
Sentiment : neutral
Entités nommées : []
New message ->je n'aime pas ça
Sentiment :  negative
Langue :  French
Entités nommées :  []
New message ->Message problématique de joelpatrick_ vers : je n'aime pas ça
New message ->Message problématique de Bot_detection vers : Message problématique de joelpatrick_ vers : je n'aime pas ça
New message ->Message problématique de Bot_detection vers : Message problématique de Bot_detection vers : Message problématique de joelpatrick_ vers : je n'aime pas ça
New message -><@889707349978259517> : je n'aime pas ça
Langue : French
Sentiment : negative
Entités nommées : []
New message ->Message problématique de Bot_detection vers : Message problématique de Bot_detection vers : Message problématique de Bot_detection vers : Message problématique de joelpatrick_ vers : je n'aime pas ça
New message ->Message problémati